In [2]:
"""
Evolve a star cluster through a galaxy, while interacting with giant molecular clouds.

Code adapted from the AMUSE textbook examples: 
https://github.com/amusecode/amuse/blob/master/examples/textbook/solar_cluster_in_semilive_galaxy.py
Notebook by Francisca Concha-Ramírez
"""

from __future__ import print_function
import math
import numpy
from amuse.lab import *
from amuse.couple import bridge
from amuse.units.optparse import OptionParser
from amuse.units import quantities

from amuse.community.galaxia.interface import BarAndSpirals3D
from amuse.community.fractalcluster.interface import new_fractal_cluster_model
from amuse.ext.composition_methods import *
from matplotlib import pyplot

In [3]:
# We will use this class for the Giant Molecular Clouds moving through the galaxy
class drift_without_gravity(object):

    def __init__(self, particles, time=0 | units.Myr):
        self.particles = particles
        self.model_time = time
        self.softening_lengths_squared = (100 | units.parsec) ** 2
        self.gravity_constant = constants.G

    def evolve_model(self, t_end):
        dt = t_end - self.model_time
        self.particles.position += self.particles.velocity * dt
        self.model_time = t_end

    def add_particles(self, p):
        self.particles.add_particles(p)

    @property
    def potential_energy(self):
        return quantities.zero

    def get_potential_at_point(self, radius, x, y, z):
        positions = self.particles.position
        result = quantities.AdaptingVectorQuantity()
        for i in range(len(x)):
            dx = x[i] - positions.x
            dy = y[i] - positions.y
            dz = z[i] - positions.z
            dr_squared = (dx * dx) + (dy * dy) + (dz * dz)
            dr = (dr_squared + self.softening_lengths_squared).sqrt()
            energy_of_this_particle = (self.particles.mass / dr).sum()
            result.append(-self.gravity_constant * energy_of_this_particle)
        return result

    def get_gravity_at_point(self, radius, x, y, z):
        positions = self.particles.position
        m1 = self.particles.mass
        result_ax = quantities.AdaptingVectorQuantity()
        result_ay = quantities.AdaptingVectorQuantity()
        result_az = quantities.AdaptingVectorQuantity()
        for i in range(len(x)):
            dx = x[i] - positions.x
            dy = y[i] - positions.y
            dz = z[i] - positions.z
            dr_squared = ((dx * dx) + (dy * dy) + (dz * dz) +
                          self.softening_lengths_squared + radius[i] ** 2)

            ax = -self.gravity_constant * (m1 * dx / dr_squared ** 1.5).sum()
            ay = -self.gravity_constant * (m1 * dy / dr_squared ** 1.5).sum()
            az = -self.gravity_constant * (m1 * dz / dr_squared ** 1.5).sum()

            result_ax.append(ax)
            result_ay.append(ay)
            result_az.append(az)
        return result_ax, result_ay, result_az

    @property
    def kinetic_energy(self):
        return (0.5 * self.particles.mass \
                * self.particles.velocity.lengths() ** 2).sum()

    def stop(self):
        return

In [ ]:
class MilkyWay_galaxy(object):

    def __init__(self, Mb=1.40592e10 | units.MSun,
                 Md=8.5608e10 | units.MSun, Mh=1.07068e11 | units.MSun):
        self.Mb = Mb
        self.Md = Md
        self.Mh = Mh

    def get_potential_at_point(self, eps, x, y, z):
        r = (x ** 2 + y ** 2 + z ** 2) ** 0.5
        R = (x ** 2 + y ** 2) ** 0.5

        # bulge
        b1 = 0.3873 | units.kpc
        pot_bulge = -constants.G * self.Mb / (r ** 2 + b1 ** 2) ** 0.5

        # disk
        a2 = 5.31 | units.kpc
        b2 = 0.25 | units.kpc
        pot_disk = -constants.G * self.Md / (R ** 2 + (a2 + (z ** 2 + b2 ** 2) ** 0.5) ** 2) ** 0.5

        # halo
        a3 = 12.0 | units.kpc
        cut_off = 100 | units.kpc
        d1 = r / a3
        c = 1 + (cut_off / a3) ** 1.02
        pot_halo = -constants.G * (self.Mh / a3) * d1 ** 1.02 / (1 + d1 ** 1.02) \
                   - (constants.G * self.Mh / (1.02 * a3)) \
                   * (-1.02 / c + numpy.log(c) \
                      + 1.02 / (1 + d1 ** 1.02) \
                      - numpy.log(1.0 + d1 ** 1.02))
        return 2 * (pot_bulge + pot_disk + pot_halo)  # multiply by 2 for
        # rigid potential

    def get_gravity_at_point(self, eps, x, y, z):
        r = (x ** 2 + y ** 2 + z ** 2) ** 0.5
        R = (x ** 2 + y ** 2) ** 0.5
        # bulge
        b1 = 0.3873 | units.kpc
        force_bulge = -constants.G * self.Mb / (r ** 2 + b1 ** 2) ** 1.5
        # disk
        a2 = 5.31 | units.kpc
        b2 = 0.25 | units.kpc
        d = a2 + (z ** 2 + b2 ** 2) ** 0.5
        force_disk = -constants.G * self.Md / (R ** 2 + d ** 2) ** 1.5
        # halo
        a3 = 12.0 | units.kpc
        d1 = r / a3
        force_halo = -constants.G * self.Mh * d1 ** 0.02 / (a3 ** 2 * (1 + d1 ** 1.02))

        ax = force_bulge * x + force_disk * x + force_halo * x / r
        ay = force_bulge * y + force_disk * y + force_halo * y / r
        az = force_bulge * z + force_disk * d * z / (z ** 2 + b2 ** 2) ** 0.5 \
             + force_halo * z / r

        return ax, ay, az

    def vel_circ(self, r):
        z = 0 | units.kpc
        b1 = 0.3873 | units.kpc
        a2 = 5.31 | units.kpc
        b2 = 0.25 | units.kpc
        a3 = 12.0 | units.kpc

        rdphi_b = constants.G * self.Mb * r ** 2 \
                  / (r ** 2 + b1 ** 2) ** 1.5
        rdphi_d = constants.G * self.Md * r ** 2 \
                  / (r ** 2 + (a2 + (z ** 2 + b2 ** 2) ** 0.5) ** 2) ** 1.5
        rdphi_h = constants.G * self.Mh * (r / a3) ** 0.02 * r \
                  / (a3 ** 2 * (1 + (r / a3) ** 1.02))

        vel_circb = rdphi_b
        vel_circd = rdphi_d
        vel_circh = rdphi_h

        return (vel_circb + vel_circd + vel_circh) ** 0.5

    def stop(self):
        return